Lets create the agentcore runtime without the starter toolkit. We first create the ECR, AgentCore Execution Role. Then we build the docker image and push it to the ECR repo

Once the image is ready for deployment, we use the Control Plane boto3 SDKs to create the Agent Runtime and Agent Runtime endpoints.

Finally we shall use the Data Plane SDK to invoke the endpoint

In [ ]:
%pip install -r requirements.txt -q --force-reinstall

In [ ]:
import os
import boto3
import random
import string
import time

# os.environ['AWS_PROFILE']='stcgenai'

session = boto3.Session()
region = session.region_name
print(f"current region: {region}")
account_id = session.client("sts").get_caller_identity()["Account"]
print(f"current account: {account_id}")


In [ ]:
agent_name = "mortgage_agent"
suffix = random.choices(string.ascii_lowercase + string.digits, k=8)
repo_name = f"{agent_name}_{''.join(suffix)}"
print(repo_name)
ecr_client = boto3.client('ecr')

repo = ecr_client.create_repository(repositoryName=repo_name)

repoArn = repo['repository']['repositoryArn']
repo_uri = repo['repository']['repositoryUri']

In [ ]:
# Load policy from external file
with open('policy.json', 'r') as f:
    policy_template = f.read()

# load trust policy from external file
with open('trust-policy.json', 'r') as f:
    trust_policy_template = f.read()

policy = policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id).replace('REPO_ARN', repoArn)
print(f"Policy loaded and updated with region: {region}, account: {account_id}, repo: {repo_name}")

trust_policy = trust_policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id)

In [ ]:
# create IAM role using the policies
iam_client = boto3.client('iam')
role_name = f"bedrock-runtime-execution-role-{''.join(suffix)}"

role = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=trust_policy
)

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='bedrock-runtime-execution-policy',
    PolicyDocument=policy
)

In [ ]:
import subprocess

auth = ecr_client.get_authorization_token()
token = auth['authorizationData'][0]['authorizationToken']
endpoint = auth['authorizationData'][0]['proxyEndpoint']

#ECR login
login_cmd = f"aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {repo_uri}"
subprocess.run(login_cmd, shell=True, check=True)


In [ ]:
# Create buildx builder (ignore if exists)
create_builder_cmd = "docker buildx create --name arm64builder --use"
subprocess.run(create_builder_cmd, shell=True, check=False) 

In [ ]:
# Build ARM64 image with custom Dockerfile
build_cmd = "docker buildx build --platform linux/arm64 --load -f Dockerfile_custom -t bedrock-agentcore-arm64 ."
subprocess.run(build_cmd, shell=True, check=True)

In [ ]:
# Tag the image
tag_cmd = f"docker tag bedrock-agentcore-arm64:latest {repo_uri}:latest"
subprocess.run(tag_cmd, shell=True, check=True)

In [ ]:
# Push to ECR
push_cmd = f"docker push {repo_uri}:latest"
subprocess.run(push_cmd, shell=True, check=True)

In [ ]:
# Create the client
client = boto3.client('bedrock-agentcore-control')

agent_name = "langgraph_mortgage_agent"
agentRuntimeName = f"{agent_name}_{''.join(suffix)}"
# Call the CreateAgentRuntime operation
response = client.create_agent_runtime(
    agentRuntimeName=agentRuntimeName,
    agentRuntimeArtifact={
        'containerConfiguration': {
            'containerUri': f"{repo_uri}:latest"
        }
    },
    networkConfiguration={"networkMode":"PUBLIC"},
    roleArn=role['Role']['Arn']
)

print(f"Status: {response['status']}")

##### Update the existing runtime with new image
```
response = client.update_agent_runtime(
    agentRuntimeId=agent_runtime_id,
    agentRuntimeArtifact={
        'containerConfiguration': {
            'containerUri': f"{repo_uri}:latest"
        }
    },
    networkConfiguration={"networkMode": "PUBLIC"}, 
    roleArn=role['Role']['Arn'] 
)
```


In [ ]:
end_status = ['READY', 'CREATE_FAILED', 'UPDATE_FAILED']
agent_runtime_id = response['agentRuntimeId']  
status = response['status'] 
while status not in end_status:
    print("Waiting for agent runtime to be ready")
    time.sleep(10)
    current_response = client.get_agent_runtime(agentRuntimeId=agent_runtime_id)
    status = current_response['status']
    print(f"Current status: {status}")

print(f"Final status: {status}")

if status == 'READY':
    print(f"Agent runtime {agent_runtime_id} is ready")
else:
    print(f"Agent runtime {agent_runtime_id} creation failed")

In [83]:
agentRuntimeArn = response['agentRuntimeArn']

endpoint_name = f"DEV_{''.join(suffix)}"

endpoint_response = client.create_agent_runtime_endpoint(
    agentRuntimeId=agent_runtime_id,
    agentRuntimeVersion=response['agentRuntimeVersion'],
    name=endpoint_name,
    description='Endpoint for mortgage agent'
)

```
endpoint_response = client.update_agent_runtime_endpoint(
    agentRuntimeId=agent_runtime_id,
    endpointName='production-endpoint',
    agentRuntimeVersion=response['agentRuntimeVersion'],
    description='Updated production endpoint'
)
```


In [ ]:
end_status = ['READY', 'CREATE_FAILED', 'UPDATE_FAILED']
endpoint_status = endpoint_response['status']

while endpoint_status not in end_status:
    print("Waiting for  runtime endpoint to be ready")
    time.sleep(10)
    endpoint_current_response = client.get_agent_runtime_endpoint(
    agentRuntimeId=agent_runtime_id,
    endpointName=endpoint_name
)
    endpoint_status = endpoint_current_response['status']
    print(f"Endpoint status: {endpoint_status}")

print(f"Final endpoint status: {endpoint_status}")

In [85]:
import json
agentcore_client = boto3.client('bedrock-agentcore')

Sample Prompts:

- Can you help me with mortgage info. Here is my customer id: 23455
- I am customer 123456 and I am considering refinancing my current mortgage. Before I make a decision, I would like to know my current mortgage details including my interest rate and remaining balance. Could you explain the main benefits and drawbacks of refinancing from a 30-year to a 15-year mortgage, and finally I am also curious about the current 15-year fixed rates to see if my mortgage repayments will be low


In [88]:

input_text = json.dumps({"prompt": "I am customer 123456 and I am considering refinancing my current mortgage. Before I make a decision, I would like to know my current mortgage details including my interest rate and remaining balance. Could you explain the main benefits and drawbacks of refinancing from a 30-year to a 15-year mortgage, and finally I am also curious about the current 15-year fixed rates to see if my mortgage repayments will be low"})

response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agentRuntimeArn,
    qualifier=endpoint_name,
    payload=input_text
)

In [ ]:
data = json.loads(response['response'].read().decode('utf-8'))
for msg in data['messages']:
    content = msg['content']
    if isinstance(content, list):
        content = ' '.join([item.get('text', str(item)) for item in content if isinstance(item, dict)])
    print(f"\n{msg['type'].upper()}: {content}")